In [2]:
%load_ext autoreload
%autoreload 2

import pickle
import pandas as pd
import os
import openai
import numpy as np
import ipdb
import re
from tqdm import tqdm
import scipy
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
import spacy
from sklearn import metrics

nlp = spacy.load("en_core_web_sm")
openai.api_key= os.environ['OPENAI_KEY']

from data_utils import *
from gpt3_utils import *
from eval_utils import *

pd.set_option('max_rows',500,'max_colwidth',10000)
pd.options.display.float_format = "{:,.2f}".format

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
results_by_prompt_sel_stratified = pickle.load(open('prompt_selection_disease_results.stratify.p','rb'))

In [4]:
#To select the best prompt for post-filtering we selected the highest recall few-shot prompt from the prompt selection experiment
#For the disease results the best recall prompt is #4

best_prompt_results = results_by_prompt_sel_stratified[4,0]

In [5]:
best_prompt_results[['sents','entities','predictions','predictions.post','bio_preds','gpt3_output_predictions']]

,sents,entities,predictions,predictions.post,bio_preds,gpt3_output_predictions
2697,The site of common side effects of sumatriptan.,[],[sumatriptan],[sumatriptan],O O O O O O O B O,"{'id': 'cmpl-4JunMCA1FZxh3myzp6bVJt43O2VOw', 'object': 'text_completion', 'created': 1640651900, 'model': 'davinci:2020-05-03', 'choices': [{'text': ' sumatriptan', 'index': 0, 'logprobs': { ""text_offset"": [ 1114, 1118, 1120, 1124, 1126, 1126, 1126, 1126 ], ""token_logprobs"": [ -0.83905035, -6.9861335e-06, -3.170517e-06, -2.2604345e-05, -0.57398665, -0.00022802629, -0.019497866, -0.5198342 ], ""tokens"": [ "" sum"", ""at"", ""ript"", ""an"", ""\n"", ""\n"", ""The"", "" site"" ], ""top_logprobs"": [ { "" sum"": -0.83905035 }, { ""at"": -6.9861335e-06 }, { ""ript"": -3.170517e-06 }, { ""an"": -2.2604345e-05 }, { ""\n"": -0.57398665 }, { ""\n"": -0.00022802629 }, { ""The"": -0.019497866 }, { "" site"": -0.5198342 } ] }, 'finish_reason': 'stop'}]}"
3709,Hemodynamic parameters and lead II electrocardiograph were monitored and recorded continuously.,[],"[lead ii electrocardiograph, electrocardiograph, electrocardiograph]","[electrocardiograph, lead ii electrocardiograph]",O O O B I I O O O O O O,"{'id': 'cmpl-4JunNy0nUgZl2WzUKQVi4zVxKeLS0', 'object': 'text_completion', 'created': 1640651901, 'model': 'davinci:2020-05-03', 'choices': [{'text': ' lead II electrocardiograph, electrocardiograph, electrocardiograph', 'index': 0, 'logprobs': { ""text_offset"": [ 1162, 1167, 1170, 1178, 1182, 1183, 1189, 1190, 1198, 1202, 1203, 1209, 1210, 1218, 1222, 1223, 1229, 1229, 1229, 1229, 1229, 1229, 1229, 1229 ], ""token_logprobs"": [ -0.7523855, -0.050331708, -0.12683891, -2.0098154e-05, -0.0005479298, -4.6445148e-05, -0.6854893, -0.778271, -0.00024513318, -5.8725527e-05, -3.5717305e-05, -0.5470452, -0.5953677, -0.00054029573, -5.848711e-05, -0.00024501586, -0.99770355, -7.648668e-05, -0.2285915, -0.06599785, -0.0002014371, -0.12761466, -0.21010391, -0.110308945 ], ""tokens"": [ "" lead"", "" II"", "" electro"", ""card"", ""i"", ""ograph"", "","", "" electro"", ""card"", ""i"", ""ograph"", "","", "" electro"", ""card"", ""i"", ""ograph"", ""\n"", ""\n"", ""H"", ""em"", ""odynamic"", "" parameters"", "" and"", "" lead"" ], ""top_logprobs"": [ { "" lead"": -0.7523855 }, { "" II"": -0.050331708 }, { "" electro"": -0.12683891 }, { ""card"": -2.0098154e-05 }, { ""i"": -0.0005479298 }, { ""ograph"": -4.6445148e-05 }, { "","": -0.6854893 }, { "" electro"": -0.778271 }, { ""card"": -0.00024513318 }, { ""i"": -5.8725527e-05 }, { ""ograph"": -3.5717305e-05 }, { "","": -0.5470452 }, { "" electro"": -0.5953677 }, { ""card"": -0.00054029573 }, { ""i"": -5.848711e-05 }, { ""ograph"": -0.00024501586 }, { ""\n"": -0.99770355 }, { ""\n"": -7.648668e-05 }, { ""H"": -0.2285915 }, { ""em"": -0.06599785 }, { ""odynamic"": -0.0002014371 }, { "" parameters"": -0.12761466 }, { "" and"": -0.21010391 }, { "" lead"": -0.110308945 } ] }, 'finish_reason': 'stop'}]}"
2724,SETTING: National Institutes of Health clinical research center.,[],[],[],O O O O O O O O O O,"{'id': 'cmpl-4JunOu3cLshH2W3P6FZt6UoxmudZ0', 'object': 'text_completion', 'created': 1640651902, 'model': 'davinci:2020-05-03', 'choices': [{'text': '', 'index': 0, 'logprobs': { ""text_offset"": [ 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131 ], ""token_logprobs"": [ -0.12271816, -0.0006408487, -0.27085453, -0.00019404522, -0.0020313787, -0.03940217, -0.0006236113, -6.170386e-05 ], ""tokens"": [ ""\n"", ""\n"", ""SET"", ""TING"", "":"", "" National"", "" Institutes"", "" of"" ], ""top_logprobs"": [ { ""\n"": -0.12271816 }, { ""\n"": -0.0006408487 }, { ""SET"": -0.27085453 }, { ""TING"": -0.00019404522 }, { "":"": -0.0020313787 }, { "" National"": -0.03940217 }, { "" Institutes"": -0.0006236113 }, { "" of"": -6.170386e-05 } ] }, 'finish_reason': 'stop'}]}"
3390,"Therefore, patients with glioblastoma sometimes have intracarotid injection of carcinostatics added to the treatment regimen.",['glioblastoma'],[glioblastoma],[glioblastoma

In [6]:
correct_entities = []
incorrect_entities = []

for true,preds in zip(best_prompt_results.entities, best_prompt_results['predictions.post']):
    true = eval(true)
    
    for pred in preds:
        if pred == '':
            continue
            
        if pred in true:
            correct_entities.append(pred)
        else:
            incorrect_entities.append(pred)
            
correct_entities = pd.DataFrame(correct_entities,columns=['phrase'])
correct_entities['tag'] = 'Yes'

incorrect_entities = pd.DataFrame(incorrect_entities,columns=['phrase'])
incorrect_entities['tag'] = 'No'

In [7]:
entities = pd.concat([correct_entities, incorrect_entities])

In [8]:
len(entities)

60

In [9]:
#Use half of the each label as "train" set (a set from which we sample in-context prompts) and half as the dev set
in_context_entities = []
dev_entities = []

for tag,g in entities.groupby('tag'):
    g = g.sample(frac=1, random_state=np.random.RandomState(42))
    in_context_entities.append(g[:int(len(g)/2)])
    dev_entities.append(g[int(len(g)/2):])

in_context_entities = pd.concat(in_context_entities)
dev_entities = pd.concat(dev_entities)

In [10]:
in_context_entities

,phrase,tag
15,hoe 140,No
19,sleepiness,No
27,dexamethasone,No
26,fentanyl,No
8,pentobarbital,No
24,antinociception,No
21,cinacalcet,No
12,attention,No
32,ergotamine / caffeine,No
9,venlafaxine overdose,No


In [ ]:
in_context_entities.to_csv('../output/bc5cdr_disease_post_processing_prompt_options.csv')

[autoreload of transformers failed: Traceback (most recent call last):
  File "/home/jimenezgutierrez.1/.local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/jimenezgutierrez.1/.local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/importlib/__init__.py", line 142, in reload
    name = module.__spec__.name
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/site-packages/transformers/__init__.py", line 2487, in __getattr__
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/site-packages/transformers/file_utils.py", line 1696, in __getattr__
    """
  File "/home/jimenezgutierrez.1/anac

[autoreload of tokenizers.models failed: Traceback (most recent call last):
  File "/home/jimenezgutierrez.1/.local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/jimenezgutierrez.1/.local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 790, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/site-packages/tokenizers/models/

[autoreload of transformers.models.mbart failed: Traceback (most recent call last):
  File "/home/jimenezgutierrez.1/.local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/jimenezgutierrez.1/.local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/importlib/__init__.py", line 142, in reload
    name = module.__spec__.name
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/site-packages/transformers/file_utils.py", line 1696, in __getattr__
    """
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/site-packages/transformers/file_utils.py", line 1696, in __getattr__
    """
  File "/home/

[autoreload of torch.overrides failed: Traceback (most recent call last):
  File "/home/jimenezgutierrez.1/.local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/jimenezgutierrez.1/.local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 790, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/site-packages/torch/overrides.py",

In [38]:
def select_prompt_entities(in_context_entity_df, num_entities_per_prompt, seed):
    assert num_entities_per_prompt % 2 == 0
    
    prompt_ents = []
    for i, g in in_context_entity_df.groupby('tag'):
        prompt_ents.append(g.sample(int(num_entities_per_prompt/2), random_state=np.random.RandomState(seed)))
        
    prompt_ents = pd.concat(prompt_ents)
    
    return prompt_ents

def create_prompt_from_ents(prompt_df, template_prompt):
    full_prompt = []
    
    for i,row in prompt_df.sample(frac=1,random_state=np.random.RandomState(42)).iterrows():
        
        ent = row[0]
        tag = row['tag']
        
        full_prompt.append(template_prompt.format(ent,tag))
        
    return '\n\n'.join(full_prompt)

In [39]:
labeled_template = 'Does the phrase "{}" refer to a disease?\n{}'
unlabeled_template = 'Does the phrase "{}" refer to a disease?\n'

In [117]:
##Calibrating Seeds
seeds = [42,0,14,21,33]

calibrate_prompt_to_entity_probs = {}

already_used_inds = []
few_shot_num = 6

for seed in seeds:
    unused_in_context_entities = in_context_entities[[i not in already_used_inds for i in in_context_entities.index]]
    
    if len(unused_in_context_entities) < few_shot_num:
        unused_in_context_entities = in_context_entities
        already_used_inds = []
        
    prompt_df = select_prompt_entities(unused_in_context_entities, few_shot_num, seed)
    already_used_inds.extend(prompt_df.index)
    
    prompt = create_prompt_from_ents(prompt_df, labeled_template)  
    calibrate_entity_probs = run_gpt3_ner_post_filtering('davinci', prompt, unlabeled_template, ['"N/A"','""','"None"'])
    
    calibrate_prompt_to_entity_probs[seed] = (prompt_df['phrase'].values, calibrate_entity_probs)

In [134]:
x = np.mean(list(calibrate_prompt_to_entity_probs[0][1].values()),axis=0)
np.diag(1/x)

array([[1.43162218, 0.        ],
       [0.        , 3.31684111]])

In [55]:
seeds = [42,0,14,21,33]

prompt_to_entity_probs = {}

already_used_inds = []
few_shot_num = 6

for seed in seeds:
    unused_in_context_entities = in_context_entities[[i not in already_used_inds for i in in_context_entities.index]]
    
    if len(unused_in_context_entities) < few_shot_num:
        unused_in_context_entities = in_context_entities
        already_used_inds = []
        
    prompt_df = select_prompt_entities(unused_in_context_entities, few_shot_num, seed)
    already_used_inds.extend(prompt_df.index)
    
    prompt = create_prompt_from_ents(prompt_df, labeled_template)
    entity_probs = run_gpt3_ner_post_filtering('davinci', prompt, unlabeled_template, dev_entities['phrase'].values)
    
    prompt_to_entity_probs[seed] = (prompt_df['phrase'].values, entity_probs)

In [56]:
for seed in seeds:
    print(prompt_to_entity_probs[seed][0])

['potassium' 'hoe 140' 'visual task' 'nausea'
 'partial loss of quadriceps functions' 'visual disturbance']
['ergotamine / caffeine' 'mtdna damage' 'sumatriptan' 'meth mouth'
 'mitochondrial abnormalities' 'glioblastoma']
['dexamethasone' 'blood pressure' 'antihyperalgesic activity' 'phlebitis'
 'convulsions' 'petechiae']
['outcome' 'ergotamine / caffeine' 'antihyperalgesic activity'
 'myelopathy' 'convulsions' 'petechiae']
['potassium' 'mtdna damage' 'oxidative phosphorylation' 'nausea' 'purpura'
 'thrombosis']


In [57]:
already_used_inds

[23, 30, 13, 9, 1, 16, 29, 11, 12, 12, 15, 11]

In [100]:
seed_stats = []
entity_df_by_seed = {}
    
for seed, entity_probs in prompt_to_entity_probs.items():
    prompt, entity_prob_dict = entity_probs
    
    entity_df = pd.DataFrame([(phrase, p[0],p[1]) for phrase, p in entity_prob_dict.items()],columns=['phrase','neg','pos'])
    
    entity_df = entity_df.merge(dev_entities,on='phrase')
    
#     for tag in ['Yes','No']:
#         tag_g = entity_df[entity_df.tag == tag]
#         display(tag_g.hist())
    
    yes_pos_avg = entity_df[entity_df.tag == 'Yes'].pos.mean()
    no_pos_avg = entity_df[entity_df.tag == 'No'].pos.mean()

    yes_pos_std = entity_df[entity_df.tag == 'Yes'].pos.min()
    no_pos_std = entity_df[entity_df.tag == 'No'].pos.max()

    entity_df['label'] = [t == 'Yes' for t in entity_df.tag]
    entity_df['pred_0.5'] = [p > 0.5 for p in entity_df.pos]
    entity_df['pred_0.4'] = [p > 0.4 for p in entity_df.pos]
    
    entity_df_by_seed[seed]  = entity_df
    
    f1_5 = metrics.f1_score(entity_df['label'], entity_df['pred_0.5'])
    prec_5 = metrics.precision_score(entity_df['label'],entity_df['pred_0.5'])
    recall_5 = metrics.recall_score(entity_df['label'],entity_df['pred_0.5'])
    
    f1_4 = metrics.f1_score(entity_df['label'], entity_df['pred_0.4'])
    prec_4 = metrics.precision_score(entity_df['label'],entity_df['pred_0.4'])
    recall_4 = metrics.recall_score(entity_df['label'],entity_df['pred_0.4'])
    
    acc_4 = metrics.accuracy_score(entity_df['label'], entity_df['pred_0.4'])
    
    seed_stats.append((seed, yes_pos_avg, no_pos_avg, yes_pos_avg - no_pos_avg, yes_pos_std, no_pos_std, f1_5, prec_5, recall_5, f1_4, prec_4, recall_4, acc_4, prompt))
        
seed_stats = pd.DataFrame(seed_stats, columns=['seed','Avg Yes Prob for Positive Phrases','Avg Yes Prob for Negative Phrases', 'Prob Difference (Pos-Neg)','Min Yes Prob in Pos Phrases', 'Max Yes Prob in Neg Phrases', '0.5 F1','0.5 Prec','0.5 Recall', '0.4 F1','0.4 Prec','0.4 Recall','0.4 Accuracy','Prompt'])

/home/jimenezgutierrez.1/anaconda3/envs/scispacy/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [101]:
seed_stats.sort_values('0.4 F1',ascending=False)

,seed,Avg Yes Prob for Positive Phrases,Avg Yes Prob for Negative Phrases,Prob Difference (Pos-Neg),Min Yes Prob in Pos Phrases,Max Yes Prob in Neg Phrases,0.5 F1,0.5 Prec,0.5 Recall,0.4 F1,0.4 Prec,0.4 Recall,0.4 Accuracy,Prompt
1,0,0.47,0.27,0.20,0.35,0.38,0.47,1.00,0.31,0.92,1.00,0.85,0.93,"[ergotamine / caffeine, mtdna damage, sumatriptan, meth mouth, mitochondrial abnormalities, glioblastoma]"
4,33,0.51,0.25,0.27,0.39,0.37,0.70,1.00,0.54,0.92,1.00,0.85,0.93,"[potassium, mtdna damage, oxidative phosphorylation, nausea, purpura, thrombosis]"
3,21,0.46,0.27,0.19,0.32,0.40,0.38,1.00,0.23,0.88,0.92,0.85,0.90,"[outcome, ergotamine / caffeine, antihyperalgesic activity, myelopathy, convulsions, petechiae]"
0,42,0.44,0.29,0.15,0.36,0.37,0.14,1.00,0.08,0.76,1.00,0.62,0.83,"[potassium, hoe 140, visual task, nausea, partial loss of quadriceps functions, visual disturbance]"
2,14,0.37,0.20,0.17,0.24,0.26,0.00,0.00,0.00,0.63,1.00,0.46,0.76,"[dexamethasone, blood pressure, antihyperalgesic activity, phlebitis, convulsions, petechiae]"


In [102]:
for seed in seeds:
    print(seed)
    entity_df = entity_df_by_seed[seed]
    display(entity_df[entity_df['label'] != entity_df['pred_0.4']])

42


,phrase,neg,pos,tag,label,pred_0.5,pred_0.4
18,proteinuria,0.61,0.39,Yes,True,False,False
19,hyperactivity,0.64,0.36,Yes,True,False,False
20,glomerulosclerosis,0.63,0.37,Yes,True,False,False
21,hepatitis,0.60,0.40,Yes,True,False,False
27,renovascular hypertension,0.60,0.40,Yes,True,False,False


0


,phrase,neg,pos,tag,label,pred_0.5,pred_0.4
18,proteinuria,0.65,0.35,Yes,True,False,False
19,hyperactivity,0.62,0.38,Yes,True,False,False


14


,phrase,neg,pos,tag,label,pred_0.5,pred_0.4
16,hepatotoxicity,0.67,0.33,Yes,True,False,False
18,proteinuria,0.62,0.38,Yes,True,False,False
19,hyperactivity,0.75,0.25,Yes,True,False,False
20,glomerulosclerosis,0.68,0.32,Yes,True,False,False
22,hypertension,0.74,0.26,Yes,True,False,False
23,renal insufficiency,0.64,0.36,Yes,True,False,False
27,renovascular hypertension,0.76,0.24,Yes,True,False,False


21


,phrase,neg,pos,tag,label,pred_0.5,pred_0.4
15,clinical use,0.60,0.40,No,False,False,True
19,hyperactivity,0.68,0.32,Yes,True,False,False
27,renovascular hypertension,0.64,0.36,Yes,True,False,False


33


,phrase,neg,pos,tag,label,pred_0.5,pred_0.4
19,hyperactivity,0.61,0.39,Yes,True,False,False
20,glomerulosclerosis,0.61,0.39,Yes,True,False,False


In [103]:
seed_stats

,seed,Avg Yes Prob for Positive Phrases,Avg Yes Prob for Negative Phrases,Prob Difference (Pos-Neg),Min Yes Prob in Pos Phrases,Max Yes Prob in Neg Phrases,0.5 F1,0.5 Prec,0.5 Recall,0.4 F1,0.4 Prec,0.4 Recall,0.4 Accuracy,Prompt
0,42,0.44,0.29,0.15,0.36,0.37,0.14,1.00,0.08,0.76,1.00,0.62,0.83,"[potassium, hoe 140, visual task, nausea, partial loss of quadriceps functions, visual disturbance]"
1,0,0.47,0.27,0.20,0.35,0.38,0.47,1.00,0.31,0.92,1.00,0.85,0.93,"[ergotamine / caffeine, mtdna damage, sumatriptan, meth mouth, mitochondrial abnormalities, glioblastoma]"
2,14,0.37,0.20,0.17,0.24,0.26,0.00,0.00,0.00,0.63,1.00,0.46,0.76,"[dexamethasone, blood pressure, antihyperalgesic activity, phlebitis, convulsions, petechiae]"
3,21,0.46,0.27,0.19,0.32,0.40,0.38,1.00,0.23,0.88,0.92,0.85,0.90,"[outcome, ergotamine / caffeine, antihyperalgesic activity, myelopathy, convulsions, petechiae]"
4,33,0.51,0.25,0.27,0.39,0.37,0.70,1.00,0.54,0.92,1.00,0.85,0.93,"[potassium, mtdna damage, oxidative phosphorylation, nausea, purpura, thrombosis]"


In [104]:
all_prompts = []

already_used_inds = []

for seed in seeds:
    unused_in_context_entities = in_context_entities[[i not in already_used_inds for i in in_context_entities.index]]
    
    if len(unused_in_context_entities) < few_shot_num:
        unused_in_context_entities = in_context_entities
        already_used_inds = []
        
    prompt_df = select_prompt_entities(unused_in_context_entities, few_shot_num, seed)
    already_used_inds.extend(prompt_df.index)
    
    prompt = create_prompt_from_ents(prompt_df, labeled_template)    
    all_prompts.append((seed,prompt, prompt_df['phrase'].values))
    
all_prompts = pd.DataFrame(all_prompts,columns=['seed','full_prompt','p'])

In [105]:
all_prompts

,seed,full_prompt,p
0,42,"Does the phrase ""potassium"" refer to a disease?\nNo\n\nDoes the phrase ""hoe 140"" refer to a disease?\nNo\n\nDoes the phrase ""visual disturbance"" refer to a disease?\nYes\n\nDoes the phrase ""visual task"" refer to a disease?\nNo\n\nDoes the phrase ""partial loss of quadriceps functions"" refer to a disease?\nYes\n\nDoes the phrase ""nausea"" refer to a disease?\nYes","[potassium, hoe 140, visual task, nausea, partial loss of quadriceps functions, visual disturbance]"
1,0,"Does the phrase ""ergotamine / caffeine"" refer to a disease?\nNo\n\nDoes the phrase ""mtdna damage"" refer to a disease?\nNo\n\nDoes the phrase ""glioblastoma"" refer to a disease?\nYes\n\nDoes the phrase ""sumatriptan"" refer to a disease?\nNo\n\nDoes the phrase ""mitochondrial abnormalities"" refer to a disease?\nYes\n\nDoes the phrase ""meth mouth"" refer to a disease?\nYes","[ergotamine / caffeine, mtdna damage, sumatriptan, meth mouth, mitochondrial abnormalities, glioblastoma]"
2,14,"Does the phrase ""dexamethasone"" refer to a disease?\nNo\n\nDoes the phrase ""blood pressure"" refer to a disease?\nNo\n\nDoes the phrase ""petechiae"" refer to a disease?\nYes\n\nDoes the phrase ""antihyperalgesic activity"" refer to a disease?\nNo\n\nDoes the phrase ""convulsions"" refer to a disease?\nYes\n\nDoes the phrase ""phlebitis"" refer to a disease?\nYes","[dexamethasone, blood pressure, antihyperalgesic activity, phlebitis, convulsions, petechiae]"
3,21,"Does the phrase ""outcome"" refer to a disease?\nNo\n\nDoes the phrase ""ergotamine / caffeine"" refer to a disease?\nNo\n\nDoes the phrase ""petechiae"" refer to a disease?\nYes\n\nDoes the phrase ""antihyperalgesic activity"" refer to a disease?\nNo\n\nDoes the phrase ""convulsions"" refer to a disease?\nYes\n\nDoes the phrase ""myelopathy"" refer to a disease?\nYes","[outcome, ergotamine / caffeine, antihyperalgesic activity, myelopathy, convulsions, petechiae]"
4,33,"Does the phrase ""potassium"" refer to a disease?\nNo\n\nDoes the phrase ""mtdna damage"" refer to a disease?\nNo\n\nDoes the phrase ""thrombosis"" refer to a disease?\nYes\n\nDoes the phrase ""oxidative phosphorylation"" refer to a disease?\nNo\n\nDoes the phrase ""purpura"" refer to a disease?\nYes\n\nDoes the phrase ""nausea"" refer to a disease?\nYes","[potassium, mtdna damage, oxidative phosphorylation, nausea, purpura, thrombosis]"


In [106]:
seed_stats = seed_stats.merge(all_prompts,on='seed')

In [108]:
seed_stats['ranking_score'] = seed_stats['0.4 F1'] + seed_stats['Prob Difference (Pos-Neg)']

In [109]:
# seed_stats.to_csv('bc5cdr_disease.post_filtering_prompt_results.tsv')